Run experiments to compare quadratic underestimators with other type of estimators.


In [49]:
import pandas as pd
import numpy as np
import interval as ival
import uvarprob as uvpr
import bnb as bnb
import sub as sub
import ivalprocessor as ivproc
import psqeprocessor as psproc
import gridsearch as gs
from sortedcontainers import SortedKeyList
from collections import namedtuple

Define an array of test files to process

In [134]:
test_files = ["/tmp/shek.csv"]

Tested methods

In [135]:
test_methods = {'QLI' : psqe_non_sym, 'QLC' : psqe_sym}

In [136]:
max_steps = 1e6
epsilon = 1e-5

In [137]:
TestResult = namedtuple('TestResult', ['nsteps', 'record_value'])

In [177]:
def psqe(prob, sym):
    psp = psproc.PSQEProcessor(rec_v = float('inf'), rec_x = None, problem = prob, eps = epsilon, global_lipint = True, use_symm_lipint = sym)
    sl = SortedKeyList(key = lambda s : s.level)
    subp = sub.Sub(0, 0, psproc.PSQEData(ival.Interval([prob.a, prob.b]),0))
    psp.compute_bounds(subp)
    sl.add(subp)
    cnt = max_steps
    steps = bnb.bnb(sl, cnt, psp)
#     print("steps performed: ", steps)
#     print("Record value = ", psp.rec_v, " at ", psp.rec_x);
#     print(sl)
    return TestResult(nsteps = steps, record_value = psp.rec_v)

In [178]:
def psqe_non_sym(prob):
    return psqe(prob, False)

In [179]:
def psqe_sym(prob):
    return psqe(prob, True)

In [180]:
def run_method(method, prob):
    result = method(prob)
    return result

In [181]:
def run_test(test):
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x)
    print(prob.min_f)
    runs = {}
    for name, method in test_methods.items():
        result = run_method(method, prob)
#         print(name, "result :", result)
        runs[name] = result
    return runs



In [182]:
def run_tests(df):
    data = {}
    for row in df.itertuples():
        for name, tres in run_test(row).items():
            steps_key = name + '_steps'
            if data.get(steps_key) == None:
                data[steps_key] = [tres.nsteps]
            else:
                data[steps_key].append(tres.nsteps)
            record_key = name + '_record'
            if data.get(record_key) == None:
                data[record_key] = [tres.record_value]
            else:
                data[record_key].append(tres.record_value)
    print(data)
    perf_df = pd.DataFrame(data, index = df.index)
    print(perf_df)
    print(perf_df.diff(axis = 1, periods = 2))

In [183]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col = 'name', comment = '#')
    return data

Main testing scenario

In [184]:
for file in test_files:
    run_tests(read_problems(file))

0.53993
1.4095
0.99992
0.41883
0.72143
0.4338
0.58908
0.51048
0.6859
0.42419
{'QLI_steps': [34132, 17300, 24144, 34798, 30838, 25137, 36476, 42372, 33404, 38037], 'QLI_record': [0.5399254341946468, 1.4095020073347462, 0.9999221948603374, 0.4188271304174497, 0.721430293535746, 0.4338045561990427, 0.5890764118896675, 0.5104762148333023, 0.6858992800492817, 0.4241880337618696], 'QLC_steps': [45433, 25564, 29484, 57474, 40140, 62005, 62499, 80981, 49128, 58150], 'QLC_record': [0.5399254341947614, 1.4095020073140274, 0.999922194859383, 0.41882713041736774, 0.7214302935357312, 0.43380455619794905, 0.5890764118897344, 0.5104762148331329, 0.6858992800483061, 0.4241880337619055]}
           QLI_steps  QLI_record  QLC_steps  QLC_record
name                                                   
rshekel_0      34132    0.539925      45433    0.539925
rshekel_1      17300    1.409502      25564    1.409502
rshekel_2      24144    0.999922      29484    0.999922
rshekel_3      34798    0.418827      57